In [ ]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time
from pythainlp.tag import tag_provinces
from pythainlp.tokenize import word_tokenize as tokenizer
from datetime import datetime
import math


In [ ]:
class spyder:
    def __init__( self ,links,base_url,depth ):
        self.base_url = base_url
        target_links={}
        for i in links:
            target_links[i]=0 
        self.target_links = target_links
        self.depth = depth
    
    def get_crawler(self):
        self.result_crawler = self.crawl(self.base_url,self.depth,0,set())
        return self.result_crawler
    
    def get_check_domain(self):
        self.check_domain_result = self.check_domain(self.base_url,self.get_crawler())
        return self.check_domain_result
    
    def get_check_not_domain(self):
        self.check_not_domain_result = self.check_not_domain(self.base_url,self.get_crawler())   
        return self.check_not_domain_result
    
    def get_check_ref(self):
        self.check_ref_result = self.check_ref(self.get_check_not_domain(),self.target_links)
        return self.check_ref_result
    
    def get_all(self):
        crawl = self.crawl(self.base_url,self.depth,0,set())
        check_domain =  self.check_domain(self.base_url,crawl) 
        check_not_domain = self.check_not_domain(self.base_url,crawl)
        check_ref = self.check_ref(check_not_domain,self.target_links)
        return check_domain,check_ref
    
    def crawl(self,url,n, depth,visited):
        if depth < n :
            visited.add(url)
            headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
            time.sleep(0.3)
            response = requests.get(url,headers=headers)
            try:
                soup = BeautifulSoup(response.text, 'html.parser')
            except:
                soup = BeautifulSoup(response.text, 'lxml')
            links = soup.find_all('a')
            links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
            links = [urljoin(url, link) for link in links if link]

            for link in links:
                if link not in visited:
                    link = link.replace(' ','')
                    visited.add(link)
                    if link.startswith(url):
                        self.crawl(link,n=n,depth=depth+1, visited=visited)
        return visited
    
    def check_domain(self,base_url,links):
        result= set()
        for link in links :
            if link.startswith(base_url):
                result.add(link)
        return result
    
    def check_not_domain(self,base_url,links):
        result= set()
        for link in links :
            if not link.startswith(base_url):
                result.add(link)
        return result
    
    def check_ref(self,links,target_links):
        for i in links:
            for j in target_links:
                if i.startswith(j):
                    target_links[j]+=1
        return target_links

In [ ]:
# cursor = conn.cursor()

# cursor.execute("DROP TABLE Temp_Link;")
# cursor.execute("DROP TABLE Domain_Link;")

# conn.commit()

In [ ]:
def update_ref():
    conn = sqlite3.connect('../Week10/inverted_index2.db')
    domain = conn.execute("SELECT domain_link FROM domain_link ;").fetchall()
    domain = [t[0] for t in domain]
    for i in domain :
        web = spyder(domain,i,1)
        ref = web.get_check_ref()
    check_link = conn.execute("SELECT link FROM documents ;").fetchall()
    check_link = [t[0] for t in check_link]
    for j in check_link:
        for k in ref:
            if j.startswith(k):
                conn.execute('UPDATE documents SET REF = ? WHERE link = ? ', (ref[k], j,))
    
update_ref()    

In [ ]:
def Crawl_to_temp(target_links,db):
    conn = sqlite3.connect(db)
    for i in target_links:
        domain = conn.execute("SELECT id FROM domain_link  WHERE domain_link  = ?", (i,)).fetchone()
        if not domain:
            conn.execute("INSERT INTO domain_link (domain_link) VALUES (?)", (i,))
        web_spyder = spyder(target_links,i,1)
        domainlinks  = web_spyder.get_check_domain()
        for link in domainlinks:
            conn.execute('''INSERT INTO Temp_link (Link) VALUES (?);''', (link,))
            conn.commit()

In [ ]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.summarize = self.get_summarize()
        self.word = self.get_word() 
    def make_sentence(self,list_word):
        list_word = [list_word]
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<10:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        return self.sentence_result
    def get_word(self):
        self.word_value = tokenizer(self.sentence, engine="newmm")
        return self.word_value
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=5)
        return self.summarize_result
    def location(self):
        self.data = self.get_tokenize()
        self.location_value = tag_provinces(self.data)
        self.Result_location = [entry for entry in self.location_value if entry[1] == 'B-LOCATION']
        return self.Result_location

In [ ]:
def spacy_process(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    
#Tokenization and lemmatization 
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    #print("Tokenize+Lemmatize:")
    #print(lemma_list)
    
    #Filter the stopword
    filtered_sentence =[] 
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    
    #Remove punctuation
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)
    #print(" ")
    #3print("Remove stopword & punctuation: ")
    #print(filtered_sentence)
    return filtered_sentence

In [ ]:
def cleansing(body):
    for i in body:
        output = i.replace('\n', '  ').replace('\xa0', '  ').replace('®', ' ').replace(';', ' ')
        output = " ".join(output.split())
    return output

In [ ]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  try:
    title_tag = soup.find('title').text
  except:
    title_tag = soup.find('title')
  try:
    body_tag = soup.find('body')
    text_below_body = body_tag.get_text().lower()
  except:
    text_below_body = 'not found'
  body_list =[]
  body_list.append(text_below_body)
  return (body_list,title_tag)


In [ ]:
def get_word(body):
    word_freq = {}
    for word in body:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1
    return word_freq

In [ ]:
from nltk import word_tokenize as thaitokenize , pos_tag, ne_chunk
from nltk import Tree

def get_continuous_chunks(text,label):
    chunked = ne_chunk(pos_tag(thaitokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree and subtree.label() == label:
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        if current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue
    if continuous_chunk ==[]:
        return 'None'

    else:
        return continuous_chunk

def eng_location(doc):
    return get_continuous_chunks(doc[0], 'GPE')

In [ ]:
def check_lang(url:str):
    data_lang,title = scrape_tags(url)
    try:
        percent = pythainlp.util.countthai(data_lang[0][0])
        if percent >50:
            thai_nlp = Thai(data_lang[0]) 
            word = thai_nlp.word
            try:
                location = 'จ.'+max(thai_nlp.get_location().keys())
            except:
                location = 'Thailand'
            new_list = [s.strip().replace('"', '') for s in word if s.strip()]
            while '' in new_list:
                new_list.remove('')
            word = get_word(new_list)
            return data_lang,word,title,location
        else:
            clean_body=cleansing(data_lang)
            body = spacy_process(cleansing(data_lang))
            word = get_word(body)
            location = eng_location(data_lang)
            return clean_body,word,title,location
    except:
        clean_body=cleansing(data_lang)
        body = spacy_process(cleansing(data_lang))
        word = get_word(body)
        location = eng_location(data_lang)
        return clean_body,word,title,location
        

In [ ]:
check_lang('https://www.thairath.co.th/news/crime/2633816')

In [ ]:
check_lang('https://www.bbc.com/news/world-asia-64957293')[3]

In [ ]:
def make_doc(link,target_links):
    link.replace(" ", "")
    d=dict()
    body,word,title,location=check_lang(link)
    d['link']= link
    d['title'] = title
    d['body']=body
    d['location']=location
    d['word'] = word
    for k in target_links:
        if link.startswith(k):
            d['ref'] = target_links[k]
    print(d)
    return d


In [ ]:
target_links = {'http://www.bbc.com':0}
for i in target_links:
    make_doc(i,target_links)

In [ ]:
def get_doc(links,n):
    target_links=dict()
    for i in links:
        target_links[i]=0
    doc=[]
    num=0
    for i in target_links:
        web_spyder=spyder(target_links,i,n)
        domain_links,target_links =web_spyder.get_all()
        print('all link =', len(domain_links))
        for j in domain_links:
            num+=1
            d = make_doc(j,target_links)
            doc.append(d)
            print(num)
    return doc

In [ ]:
target_links=['https://www.bbc.com/news','https://edition.cnn.com','https://www.bangkokpost.com','http://www.thairath.co.th/news',]


In [ ]:
doc = get_doc(target_links,3)

In [ ]:
import sqlite3

# Connect to SQLite3 database
conn = sqlite3.connect('../Week10/inverted_index2.db')
cursor = conn.cursor()

# Create tables for words, documents, and word frequencies

conn.execute('''
CREATE TABLE words (
    ID INTEGER PRIMARY KEY,
    Word TEXT NOT NULL UNIQUE
);
''')

conn.execute('''
CREATE TABLE documents (
    ID INTEGER PRIMARY KEY,
    Link TEXT NOT NULL UNIQUE ,
    Title TEXT,
    Body TEXT,
    Location TEXT,
    Ref INT,
    Time TEXT
);
''')

conn.execute('''
CREATE TABLE word_frequencies (
    Word_ID INTEGER ,
    Doc_ID INTEGER ,
    Frequency INTEGER NOT NULL,
    TF_IDF REAL ,
    PRIMARY KEY (word_id, doc_id),
    FOREIGN KEY (word_id) REFERENCES words(id),
    FOREIGN KEY (doc_id) REFERENCES documents(id)
);
''')

In [ ]:
def update_tf_idf():
    conn = sqlite3.connect('../Week10/inverted_index2.db',timeout=3)

    cursor = conn.execute('SELECT COUNT(*) FROM documents')
    N = cursor.fetchone()[0]
    
    cursor = conn.execute('SELECT ID, Word FROM words')
    words = cursor.fetchall()
    
    for word in words:
        word_id = word[0]
        word_str = word[1]

        cursor = conn.execute('SELECT Doc_ID, Frequency FROM word_frequencies WHERE Word_ID = ?', (word_id,))
        doc_freqs = cursor.fetchall()

        df = len(doc_freqs)
        idf = math.log(N / df)

        for doc_freq in doc_freqs:
            doc_id = doc_freq[0]
            tf = doc_freq[1]
            tf_idf = tf * idf
            conn.execute('UPDATE word_frequencies SET TF_IDF = ? WHERE Word_ID = ? AND Doc_ID = ?', (tf_idf, word_id, doc_id))

    conn.commit()

In [ ]:
def insert_to_database(doc):
  conn = sqlite3.connect('../Week10/inverted_index2.db')
  for i in doc:
    conn.execute('''INSERT INTO documents (Link, Title, Body, Location, Ref, Time) VALUES (?, ?, ?, ?, ?, ?);''', (str(i['link']), str(i['title']), str(i['body']), str(i['location']), int(i['ref']), datetime.now()))
    doc_id = conn.execute("SELECT last_insert_rowid()").fetchone()[0]
    
    for j in i['word'].keys():
      word_id = conn.execute("SELECT id FROM words WHERE word = ?", (j,)).fetchone()
      if not word_id:
        conn.execute("INSERT INTO words (word) VALUES (?)", (j,))
        word_id = conn.execute("SELECT last_insert_rowid()").fetchone()[0]
      else:
        word_id = word_id[0]
      
      conn.execute('''INSERT INTO word_frequencies (word_id, doc_id, Frequency) VALUES (?, ?, ?);''', (word_id, doc_id, i['word'][j]))
  
 
    
  conn.commit()
  update_tf_idf()


In [ ]:
insert_to_database(doc)

In [ ]:
def delete_data(link):
    db_dir = '../Week10/inverted_index2.db'
    conn = sqlite3.connect(db_dir,timeout=10)
    doc_id = conn.execute('''
    SELECT id FROM documents WHERE link = ?; ''', (link,)).fetchone()[0]
    conn.execute('''
        DELETE FROM documents WHERE link = ?; ''', (link,))

    conn.execute('''
        DELETE FROM word_frequencies WHERE Doc_ID = ?;''', (doc_id,))

    conn.execute('''
        DELETE FROM words
        WHERE NOT EXISTS (SELECT 1 FROM word_frequencies WHERE word_frequencies.word_id = words.id );''')
    
    conn.commit()
    update_tf_idf()


In [ ]:
{'link':'wwww.','title':'asdasd','body':'asdasd',}

In [ ]:
def update_data(link):
    db_dir = '../Week10/inverted_index2.db'
    conn = sqlite3.connect(db_dir,timeout=10)
    target_links={}
    for i in link:
        target_links[link]=0
    for i in target_links:
        get_link = spyder(target_links,i,2)
        domain_link,target_links = get_link.get_all()
    for j in domain_link:
        link = conn.execute('''SELECT  documents.link
                                    FROM documents
                                    WHERE documents.link = ?
                                    ''',(j,)) 
        link = link.fetchone()
        doc = [make_doc(j,target_links)]
        if link == None :
            print(j)
            insert_to_database(doc)
        else:
            print(j)
            delete_data(j)
            insert_to_database(doc)
    conn.close()

In [ ]:
update_data('https://www.bbc.com/news',3)

In [ ]:
sentence = [str(input())]
clean_sentence = cleansing(sentence)
word = spacy_process(clean_sentence)
print(clean_sentence)
print(word)

In [ ]:
def search():
    conn = sqlite3.connect('../Week10/inverted_index2.db')
    cursor = conn.cursor()
    search_term = [str(input())]
    clean_sentence = cleansing(search_term)
    words = spacy_process(clean_sentence)
    doc_lists = []
    for word in words:
        cursor.execute('SELECT Doc_ID FROM word_frequencies WHERE Word_ID = (SELECT ID FROM words WHERE Word = ?)', (word,))
        doc_lists.append(set(doc[0] for doc in cursor.fetchall()))

    # Find the documents that contain all the words in the sentence
    matching_docs = set.intersection(*doc_lists)

    # Calculate the score for each document
    scores = {}
    for doc_id in matching_docs:
        score = sum(cursor.execute('SELECT Frequency FROM word_frequencies WHERE Word_ID = (SELECT ID FROM words WHERE Word = ?) AND Doc_ID = ?', (word, doc_id)).fetchone()[0] for word in words)
        scores[doc_id] = score

    # Sort the documents by score in descending order
    sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    # Get the list of document links
    doc_links = []
    for doc_id, score in sorted_docs:
        cursor.execute('SELECT Link, title FROM documents WHERE ID = ?', (doc_id,))
        doc_links.append(cursor.fetchone()[0])
        
    
            

    # Close the database connection and return the list of document links
    conn.close()
    print(search_term)
    return doc_links

In [ ]:
search()

In [ ]:
import sqlite3
import math

def search_sentence():
    conn = sqlite3.connect('../Week10/inverted_index2.db')
    cursor = conn.cursor()
    search_term = str(input())
    list_term = [search_term]
    clean_sentence = cleansing(list_term)
    words = spacy_process(clean_sentence)

    # Retrieve the word IDs for each word in the sentence
    word_ids = []
    for word in words:
        cursor.execute('SELECT word_ID FROM words WHERE word = ?', (word,))
        result = cursor.fetchone()
        if result:
            word_ids.append(result[0])

    # Join the word_frequencies and documents tables to get the documents that contain any of the words in the sentence
    cursor.execute('''
        SELECT documents.Link, documents.Title, SUM(word_frequencies.TF_IDF)
        FROM word_frequencies
        JOIN documents ON documents.ID = word_frequencies.Doc_ID
        WHERE word_frequencies.Word_ID IN ({})
        GROUP BY documents.ID
        ORDER BY SUM(word_frequencies.TF_IDF) DESC
    '''.format(','.join(str(id) for id in word_ids)))

    # Get the document links and titles for the top results
    results = cursor.fetchall()
    print('search term : ',search_term)
    print('search result: ')
    for result in results:
        print(result[0], result[1])


In [ ]:
search_sentence()

In [ ]:
import sqlite3
import math

def sentence_search():
    conn = sqlite3.connect('../Week10/inverted_index2.db')
    cursor = conn.cursor()

    # Split the query into individual words
    search_term = [str(input())]
    clean_sentence = cleansing(search_term)
    words = spacy_process(clean_sentence)

    # Retrieve the documents that contain each word
    doc_lists = []
    for word in words:
        cursor.execute("SELECT Doc_ID, TF_IDF FROM word_frequencies JOIN words ON words.ID = word_frequencies.word_ID WHERE word = ?", (word,))
        doc_list = cursor.fetchall()
        doc_lists.append(doc_list)

    # Merge the document lists using the TF-IDF scores
    doc_scores = {}
    for doc_list in doc_lists:
        for doc_id, tf_idf in doc_list:
            if doc_id in doc_scores:
                doc_scores[doc_id] += tf_idf
            else:
                doc_scores[doc_id] = tf_idf

    # Rank the documents by their overall relevance
    ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)

    # Retrieve the links and titles of the top documents
    results = []
    for doc_id, score in ranked_docs:
        cursor.execute("SELECT Link, Title FROM documents WHERE ID = ?", (doc_id,))
        link, title = cursor.fetchone()
        results.append((link, title))

    conn.close()

    return results


In [ ]:
a =sentence_search()
for i in a :
    print (i)